# Homework 4: Practical analysis with BioPython

For the homework, you are going to extend the code from the analysis of our FASTQ file in lectures 8 and 9.
Recall that the FASTQ file contains reads from a real sequencing run of influenza virus HA and NA genes.

The _actual sequences_ are as follows:

    5'-[end of HA]-AGGCGGCCGC-[16 X N barcode]-3'
    
or 

    5'-[end of NA]-AGGCGGCCGC-[16 X N barcode]-3'
    
The end of NA is:

    ...CACGATAGATAAATAATAGTGCACCAT
    
The end of HA is:

    ...CCGGATTTGCATATAATGATGCACCAT
    
The _sequencing reads_ (located in the FASTQ file) are from the reverse end of these actual sequences, so the first thing in the sequencing reads is the reverse complement of the barcode followed by the reverse complement of the constant sequence, etc.
The reads can originate from **either** HA or NA, and that will be distinguished by the most 3' end of the read.
But in our example exercise in class, we did not distinguish among reads matching to HA and NA, as we didn't even look far enough into the read to tell the identity.

For the homework, your goal is to write code that extends the material from lectures 8 and 9 to also distinguish between HA and NA.
This homework can be completed almost entirely by re-using code from lecture 9. You will need to set up your analysis to do the following:
 1. Get the reverse complement of each read.
 2. Determine if it matches the expected pattern for HA and NA, and if so which one.
 3. If it matches, extract the barcode and add it to a dictionary to keep track of counts.
 4. Determine the number and distribution of barcodes for HA and NA separately.

Please include code to address each of the following questions. Please include code comments to explain what your code is attempting to accomplish. Don't forget to include references to the sources you used to obtain your answer, including your classmates (if you are working in groups).  

1. How many reads map to HA, and how many reads map to NA?

In [43]:
import re
import Bio.SeqIO

# Read in the sequencing reads
seqreads = list(Bio.SeqIO.parse('barcodes_R1.fastq', format='fastq'))

# Convert the sequencing reads into strings
seqreads_str = []
for seqrecord in seqreads:
    sequence = str(seqrecord.seq) # convert sequence part to string
    seqreads_str.append(sequence) # add string sequence to list

# define a function to get the reverse compliment of the DNA sequence
def reverse_complement(seq, unk_partner = 'N'):
    base_partner = {'A':'T', 'T':'A', 'C':'G', 'G':'C'}
    rseq = ''
    for a in seq:
        if a in base_partner:
            # look up the complementary base in the dictionary
            pair = base_partner[a]
            rseq = pair + rseq
        else:
            rseq = unk_partner + rseq
    return rseq

# turn the reverse compliment into a list
rev = []
for seq in seqreads_str:
    reverse_complement(seq)
    rev.append(reverse_complement(seq))

' '.join(rev)

# Determine if a read maps to HA or NA
def read_virus(seqs, end_len = 27, downstream='AGGCGGCCGC', bclen = 16):
    
    # compile the search pattern
    virus_patern = re.compile('(?P<virus_type>[ATCG]{' + str(end_len) + '})'+ downstream + f"(?P<barcode>[ATCGN]{{{bclen}}})$")

    # set your virus types
    NA = 'CACGATAGATAAATAATAGTGCACCAT'
    HA = 'CCGGATTTGCATATAATGATGCACCAT'

    # make an empty dictionary to count your virus types
    virus_count = {'NA_count':0, 'HA_count':0, 'neither':0} 

    for seq in seqs: # iterate through all reads and search for the pattern
        match = virus_patern.search(seq)

        if match:
            virus_type = match.group("virus_type")
            if virus_type == NA:
                virus_count['NA_count'] +=1
            elif virus_type == HA:
                virus_count['HA_count'] += 1
            else:
                virus_count['neither'] += 1
    return virus_count


read_virus(rev)


{'NA_count': 3910, 'HA_count': 5246, 'neither': 248}

2. What is the HA barcode with the most counts (and how many counts)? Also answer the same question for NA.

    _Hint: you will need to find the key associated with the maximum value in your dictionary. There are many ways to do this._

In [67]:
# Here I'm only assessing HA reads, the NA reads are on the next cell. 
# I know I could probably do it in one cell, but I've been doing this HW for 4 hours... please have mercy

# define a function to identify your HA reads
def read_barcode(seqread, bclen = 16, end_len = 27, upstream='AGGCGGCCGC'):
    # get the reverse complement of the read
    reverse = reverse_complement(seqread)
    
    # compile the barcode search pattern
    HA_barcode_pattern = re.compile('CCGGATTTGCATATAATGATGCACCAT' + upstream + f"(?P<barcode>[ATCGN]{{{bclen}}})$")
    
    # search for the barcode pattern
    match = HA_barcode_pattern.search(reverse)
    
    if match:
        barcode = match.group('barcode')
        return barcode
    else:
        return None

# get the counts for all barcodes 
HA_barcode_counts = {}

for seq in seqreads_str: # iterate through all reads
    barcode = read_barcode(seq)
    if barcode: # if there is a valid barcode, add it to the dictionary
        if barcode in HA_barcode_counts:
            HA_barcode_counts[barcode] += 1
        else:
            HA_barcode_counts[barcode] = 1

# now that we have obtained the barcode counts, we can select the key with the max value

max_barcode_HA = max(HA_barcode_counts, key=HA_barcode_counts.get)
print('The HA barcode with the most counts is:', max_barcode_HA)

The HA barcode with the most counts is: CCCGACCCGACATTAA and it has 155 counts


In [68]:
# Now the same but for NA 

# define a function to identify your NA reads
def read_barcode(seqread, bclen = 16, end_len = 27, upstream='AGGCGGCCGC'):
    # get the reverse complement of the read
    reverse = reverse_complement(seqread)
    
    # compile the barcode search pattern
    NA_barcode_pattern = re.compile('CACGATAGATAAATAATAGTGCACCAT' + upstream + f"(?P<barcode>[ATCGN]{{{bclen}}})$")
    
    # search for the barcode pattern
    match = NA_barcode_pattern.search(reverse)
    
    if match:
        barcode = match.group('barcode')
        return barcode
    else:
        return None

# get the counts for all barcodes 
NA_barcode_counts = {}

for seq in seqreads_str: # iterate through all reads
    barcode = read_barcode(seq)
    if barcode: # if there is a valid barcode, add it to the dictionary
        if barcode in NA_barcode_counts:
            NA_barcode_counts[barcode] += 1
        else:
            NA_barcode_counts[barcode] = 1

# now that we have obtained the barcode counts, we can select the key with the max value

max_barcode_NA = max(NA_barcode_counts, key=NA_barcode_counts.get)
max_barcode_count_NA = max(NA_barcode_counts.values())
print('The NA barcode with the most counts is:', max_barcode_NA, 'and it has', max_barcode_count_NA, 'counts')

The NA barcode with the most counts is: ACCAGTTCTCCCCGGG and it has 152 counts
